Byte-level Byte-Pair Encoding (BBPE) is a byte-level variant of Byte-Pair Encoding (BPE). Despite the word 'byte' in its name, BPE does not actually decompose each word into bytes. It instead, breaks down the words into characters. A large drawback however is that if the characters are represented using UTF-8, then there are 130,000 possible Unicode representations. However, in a typical corpus, it may be that only 30,000-60,000 Unicode characters are actually present. Therefore, a base vocabulary set of all 130,000 Unicode characters is memory expensive, and using a smaller base set removes the ability to generalize to characters outside of this base set.

BBPE tackles this by going a level further, and decomposing characters into bytes, e.g., a character represented by 3 bytes is separated into 3 bytes and added to the vocabulary. Additionally, the initial vocabulary set is only 256, ($2^4 = 256$, the number of combinations of a single byte). This allows any Unicode to be recreated as it is a sequence of one to four bytes. However, there are only 130,000 Unicode assignments, far less that what can be represented by $2^{32}$ bits. Further, once broken into bytes, the BPE algorithm will merge them into tokens, but it may be that a character needs to be represented by multiple tokens. For example, suppose a waving hand emoji is represented by four bytes, `[B1], [B2], [B3], [B4]`. After running BBPE, we may have merged `[B1], [B2], [B3] -> [B123]`, but `[B4]` is left alone. This might be okay because after training, it the LLM may place the sequence of tokens, `[B123]` before `[B4]` with high probability, however, in rare scenarios it may pair tokens `[B123], [C_1]` where `[C_1]` is some other token, and this combination does not produce a valid Unicode character.

Since tokens are ultimately sequences of bytes, it is not necessarily the case that the output of an auto-regressive NLP model will be tokens that are immediately convertible to Unicode characters. Instead, one convert the output stream of tokens into a byte-stream, and thereafter, use a recursive approach to find the optimal grouping of bytes that lends the most reconstructible Unicode characters. This notebook will explore algorithm 1 from ["Wang et al."](https://arxiv.org/abs/1909.03341) which does exactly that.

For a given byte sequence $\left\{ B \right\}_{k=1}^N$, we denote the maximum number of characters that we can recover from it as $f(k)$. Then $f(k)$ has optimal substructure and can be solved by dynamic programming:
$$
f(k) = \max_{t=1,2,3,4} \left\{ f(k - t) + g(k-t+1, k) \right\}
$$
where $g(i,j)=1$ if $\{B\}_{k=i}^j$ corresponds to a valid character, otherwise 0. In addition to solving for $k$, the selections at position $k$ are recording so that the solution may be recovered via back-tracking. $t$ belongs to the set of numbers $\{1, 2, 3, 4\}$ because a unicode character may be composed of 1 to 4 bytes. The design of UTF-8 encoding also allows for a unique recovery process which ensures that for a character with multiple bytes, its trailing bytes **will not** make a valid UTF-8 encoded character. For example, if our byte sequence is four bytes, and these four bytes together represent a Unicode character, then $f(4) = 1$, and cannot be larger. In other words, none of the individual bytes, pairings, or triplets, will produce a valid Unicode character. Only the four bytes together will produce the Unicode character.

UTF-8 has precise bit-pattern rules:

| Sequence length |                              Pattern |
|:----------------|-------------------------------------:|
| 1 byte          |                             0xxxxxxx |
| 2 bytes         |                    110xxxxx 10xxxxxx |
| 3 bytes         |           1110xxxx 10xxxxxx 10xxxxxx |
| 4 bytes         |  11110xxx 10xxxxxx 10xxxxxx 10xxxxxx |

This means that it is possible to manually determine if a list of bytes represents a Unicode character in a deterministic fashion.

In [6]:
from typing import *
from transformers import GPT2Tokenizer
import random

In [7]:
def is_valid_pattern_utf8(b: List[int]):
    """
    UTF-8 has precise bit-pattern rules:

    +-----------------+-----------------------------------------------+
    | Sequence length | Pattern                                       |
    +=================+===============================================+
    | 1 byte          | 0xxxxxxx                                      |
    +-----------------+-----------------------------------------------+
    | 2 bytes         | 110xxxxx 10xxxxxx                             |
    +-----------------+-----------------------------------------------+
    | 3 bytes         | 1110xxxx 10xxxxxx 10xxxxxx                    |
    +-----------------+-----------------------------------------------+
    | 4 bytes         | 11110xxx 10xxxxxx 10xxxxxx 10xxxxxx           |
    +-----------------+-----------------------------------------------+

    This means that it is possible to manually determine if a list of bytes represents a Unicode character in a deterministic fashion. This method does exactly that.


    :param b:   List of 1 to 4 bytes representing a potentially valid Unicode character.
    :return:    Boolean indicating if the bytes encoded a valid Unicode character.
    """
    b = list(b)
    n = len(b)

    assert 1 <= n <= 4, f"List of bytes must have length in range [1, 4]. Has length {n}."

    if n == 1:
        return b[0] < 0x80

    if n == 2:
        return (0xC0 <= b[0] <= 0xDF) and (0x80 <= b[1] <= 0xBF)

    if n == 3:
        return (0xE0 <= b[0] <= 0xEF) and \
               (0x80 <= b[1] <= 0xBF) and \
               (0x80 <= b[2] <= 0xBF)

    if n == 4:
        return (0xF0 <= b[0] <= 0xF4) and \
               (0x80 <= b[1] <= 0xBF) and \
               (0x80 <= b[2] <= 0xBF) and \
               (0x80 <= b[3] <= 0xBF)

    return False

from typing import List

def is_valid_utf8(b: List[int]) -> bool:
    """
    Fully validates a UTF-8 sequence including:
    - Correct leading-byte patterns
    - Correct continuation-byte patterns
    - No overlong encodings
    - No surrogate code points
    - No code points > U+10FFFF
    """

    b = list(b)
    n = len(b)

    if not (1 <= n <= 4):
        return False

    # ------------------------------------------------------------------
    # 1) Validate structure (bit patterns)
    # ------------------------------------------------------------------

    # 1-byte (ASCII)
    if n == 1:
        if b[0] > 0x7F:
            return False
        codepoint = b[0]

    # 2-byte
    elif n == 2:
        if not (0xC2 <= b[0] <= 0xDF):        # C0/C1 are invalid (overlong)
            return False
        if not (0x80 <= b[1] <= 0xBF):
            return False
        codepoint = ((b[0] & 0x1F) << 6) | (b[1] & 0x3F)

    # 3-byte
    elif n == 3:
        b0 = b[0]
        if not (0xE0 <= b0 <= 0xEF):
            return False

        # E0 must have second byte >= A0 (avoid overlongs)
        if b0 == 0xE0 and not (0xA0 <= b[1] <= 0xBF):
            return False
        # ED must have second byte <= 9F (avoid surrogates)
        if b0 == 0xED and not (0x80 <= b[1] <= 0x9F):
            return False

        if not (0x80 <= b[1] <= 0xBF and 0x80 <= b[2] <= 0xBF):
            return False

        codepoint = ((b[0] & 0x0F) << 12) | ((b[1] & 0x3F) << 6) | (b[2] & 0x3F)

    # 4-byte
    elif n == 4:
        b0 = b[0]
        if not (0xF0 <= b0 <= 0xF4):          # F5–F7 disallowed
            return False

        # F0 must have second byte >= 0x90 (avoid overlong)
        if b0 == 0xF0 and not (0x90 <= b[1] <= 0xBF):
            return False
        # F4 must have second byte <= 0x8F (avoid > U+10FFFF)
        if b0 == 0xF4 and not (0x80 <= b[1] <= 0x8F):
            return False

        if not (0x80 <= b[1] <= 0xBF and 0x80 <= b[2] <= 0xBF and 0x80 <= b[3] <= 0xBF):
            return False

        codepoint = ((b[0] & 0x07) << 18) | ((b[1] & 0x3F) << 12) | \
                    ((b[2] & 0x3F) << 6) | (b[3] & 0x3F)

    # ------------------------------------------------------------------
    # 2) Semantic checks on decoded scalar value
    # ------------------------------------------------------------------

    # UTF-16 surrogate range is forbidden
    if 0xD800 <= codepoint <= 0xDFFF:
        return False

    # Maximum valid Unicode scalar value
    if codepoint > 0x10FFFF:
        return False

    return True


# testing the above method:
tests = [
    b"a",             # valid (ASCII character)
    b"\xc3\xa9",      # valid (é)
    b"\xf0\x9f\x98\x80",  # valid (😀)
    b"\xf0\x28\x8c\xbc",  # invalid UTF-8
]

for t in tests:
    is_valid = is_valid_utf8(t)
    if is_valid:
        decoded_ret = t.decode('utf-8')
    else:
        decoded_ret = "'invalid unicode representation'"
    print(f"{t}, decoded result: {decoded_ret}")

b'a', decoded result: a
b'\xc3\xa9', decoded result: é
b'\xf0\x9f\x98\x80', decoded result: 😀
b'\xf0(\x8c\xbc', decoded result: 'invalid unicode representation'


In [8]:
def get_maximal_string(byte_str: List[int]) -> str:
    """

    :param byte_str:
    :return:
    """

    n = len(byte_str)
    solutions = [(0, 1) for _ in range(n)]

    def _recurse(k: int) -> int:
        """

        :param k:
        :return:
        """

        if k < 0:
            # invalid index position
            return 0

        if k == 0:
            # Return whether the byte at this position is a Unicode character or not.
            # The best_t is -1 since we cannot move backwards from this position.
            best_solution = int(is_valid_utf8([byte_str[k]]))
            solutions[0] = (best_solution, 1)
            return best_solution

        best_solution = 0
        best_t = 1
        for t in range(1, 5):

            if k - t + 1 < 0:
                # this start index is out of range in our byte string
                break

            # extract the byte sequence of length t and check if it is a valid Unicode
            seq = byte_str[k - t + 1 : k + 1]
            valid = is_valid_utf8(seq)

            if not valid:
                # Invalid UTF-8 can only consume 1 byte
                if t != 1:
                    continue
                curr_solution = _recurse(k - 1)
            else:
                # Valid UTF-8 consuming t bytes
                curr_solution = _recurse(k - t) + 1

            if curr_solution > best_solution:
                best_solution = curr_solution
                best_t = t

        # store the results for backtracking
        solutions[k] = (best_solution, best_t)

        return best_solution

    # Process our stream of bytes.
    _recurse(n - 1)
    # Backtrack from the last byte, we will include it even if it is invalid.
    bt_k = n - 1

    solution_list = [] # stores our valid unicode characters in reverse order
    while bt_k > -1:
        # unpack the best value of t for this subproblem
        _, curr_t = solutions[bt_k]
        # use this value to get the current byte sequence at this subproblem
        byte_seq = byte_str[bt_k - curr_t + 1:bt_k + 1]
        if is_valid_utf8(byte_seq):
            # This is a valid sequence of bytes we can decode into a UTF-8 character.
            curr_char = bytes(byte_seq).decode('utf-8')
        else:
            # Our optimal sequence has an invalid Unicode character, display  by default.
            curr_char = "\uFFFD"
        # append our character
        solution_list.append(curr_char)
        # advance backwards with respect to our best value of t
        bt_k -= curr_t

    return "".join(solution_list[::-1])

In [9]:
# test our above recursive method
subset_bytes = [
    b"a",                 # valid ASCII
    b"\xc3\xa9",          # valid UTF-8 for "é"
    b"\xf0\x9f\x98\x80",  # valid UTF-8 for 😀
]

# -------------------------------
# Helpers to generate UTF-8-style bytes
# -------------------------------

def gen_leading_byte(choices: Tuple[int] = (1, 2, 3, 4)):
    """
    Generate a UTF-8 leading byte (0xxxxxxx, 110xxxxx, 1110xxxx, 11110xxx).
    Each leading byte expects 0, 1, 2, and 3 continuing bytes, respectively.
    """
    choice = random.choice(choices)
    if choice == 1:
        return random.randrange(0x00, 0x80)          # 0xxxxxxx
    elif choice == 2:
        return random.randrange(0xC0, 0xE0)          # 110xxxxx
    elif choice == 3:
        return random.randrange(0xE0, 0xF0)          # 1110xxxx
    else:
        return random.randrange(0xF0, 0xF8)          # 11110xxx

def gen_cont_byte():
    """
    Generate a continuation byte (10xxxxxx).
    Such continuation bytes are expected to follow after a leading byte in the
    Unicode standard.
    """
    return random.randrange(0x80, 0xC0)

def has_valid_proper_prefix(seq: bytes) -> bool:
    """Return True if any proper prefix of seq is a valid UTF-8 sequence."""
    for L in range(1, len(seq)):
        if is_valid_utf8(list(seq[:L])):
            return True
    return False

def gen_invalid_utf8(max_attempts: int = 100):
    """
    Generate a sequence of length 1–4 where:

    - Each byte follows UTF-8 structural rules.
    - The sequence is globally an invalid UTF-8. We ensure this by appending the wrong number of continuation bytes.
    """
    for _ in range(max_attempts):
        length = random.randint(1, 4)

        # Case 1: length == 1 → generate a continuation byte, which is invalid alone
        if length == 1:
            return bytes([gen_leading_byte((3,))]), 1
            # return bytes([gen_cont_byte()])  # e.g. b'\x90'

        # Case 2: invalid because the number of continuation bytes is wrong
        lead = gen_leading_byte()

        # Determine expected continuation bytes
        if 0xC0 <= lead <= 0xDF:   # 110xxxxx → expects 1 continuation byte
            expected = 1
        elif 0xE0 <= lead <= 0xEF: # 1110xxxx → expects 2
            expected = 2
        else:                      # 11110xxx → expects 3
            expected = 3

        # Corrupt the Unicode by generating the wrong number of continuation bytes that the
        # leading byte expects. The total length will be in the range 1 through 4.
        num_cont_bytes = (expected % 3) + 1
        gen_list = [lead] + [gen_cont_byte() for _ in range(num_cont_bytes)]
        seq = bytes(gen_list)
        if is_valid_utf8(seq) or has_valid_proper_prefix(seq):
            continue
        return seq, len(gen_list)

    return bytes([gen_cont_byte()]), 1

def approx_random_bytes(subset, n):
    result = b""
    labels = []

    for i in range(n):
        # Choose between:
        #  - valid UTF-8 from provided subset
        #  - invalid UTF-8 sequence
        if random.random() < 0.9:
            # Case 1: choose valid char
            curr_choice = random.choice(subset)
            result += curr_choice
            labels.append(curr_choice.decode("utf-8"))
            # labels[i] = curr_choice.decode("utf-8")
        else:
            # Case 2: generate invalid UTF-8
            curr_choice, num_bytes = gen_invalid_utf8()
            result += curr_choice
            labels.extend(["\uFFFD"]*num_bytes)
            # labels[i] = "\uFFFD"

    return result, "".join(labels)

k = 5 # generate k random samples of bytes with length n
n = 5 # number of choices in each sequence
tests = [
    approx_random_bytes(subset_bytes, n) for _ in range(k)
]
tests.append(
    # test case where the last byte is invalid
    (b'a\xf0\x9f\x98\x80\xc3\xa9a\xe1', "a😀éa�")
)
for test, label in tests:
    result = get_maximal_string(test)
    test_passed = result == label
    print(f"{test}, \nground_truth label: {label}\ndecoded result: {result}")
    print(f"test {'passed' if test_passed else 'failed'}\n")


b'\xf0\x9f\x98\x80\xf0\x9f\x98\x80\xc3\xa9aa', 
ground_truth label: 😀😀éaa
decoded result: 😀😀éaa
test passed

b'\xf0\x9f\x98\x80\xea\xc3\xa9aa', 
ground_truth label: 😀�éaa
decoded result: 😀�éaa
test passed

b'\xf0\x9f\x98\x80a\xf0\x9f\x98\x80\xf1\xa3a', 
ground_truth label: 😀a😀��a
decoded result: 😀a😀��a
test passed

b'aa\xc3\xa9\xc3\xa9a', 
ground_truth label: aaééa
decoded result: aaééa
test passed

b'\xf0\x9f\x98\x80a\xf0\x9f\x98\x80aa', 
ground_truth label: 😀a😀aa
decoded result: 😀a😀aa
test passed

b'a\xf0\x9f\x98\x80\xc3\xa9a\xe1', 
ground_truth label: a😀éa�
decoded result: a😀éa�
test passed



In [19]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
text = "Hello, world! 😀"
token_ids = tokenizer.encode(text)
d = tokenizer.decode(token_ids)
# Convert each token ID to string
tokens_as_str = [tokenizer.convert_ids_to_tokens(tid) for tid in token_ids]
print("".join(tokens_as_str)) # display the tokens as a string

# Convert token strings to bytes
tokens_as_bytes = [t.encode('utf-8') for t in tokens_as_str]

# Flatten the bytes into a list of integers
flat_bytes = [b for token in tokens_as_bytes for b in token]

# Reconstruct my byte-level function
decoded_text = get_maximal_string(flat_bytes)
print(decoded_text)  # should match string as above

Hello,Ġworld!ĠðŁĺĢ
Hello,Ġworld!ĠðŁĺĢ
